In [12]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
import os

In [ ]:
import pandas as pd

# Read the training and test datasets from CSV files with specified encoding
train_df = pd.read_csv('train.csv', encoding='ISO-8859-1')
test_df = pd.read_csv('test.csv', encoding='ISO-8859-1')


In [ ]:
# Remove rows with missing values in 'text' or 'sentiment' columns from both datasets
train_df = train_df.dropna(subset=['text', 'sentiment'])
test_df = test_df.dropna(subset=['text', 'sentiment'])


In [7]:
len(train_df), len(test_df)

(27480, 3534)

In [ ]:
import pandas as pd

# Create a dictionary to map sentiment strings to numeric labels
label_mapping = {sent: idx for idx, sent in enumerate(['negative', 'neutral', 'positive'])}

# Select relevant columns and apply label mapping for training data
train_data = train_df[['text', 'sentiment']].copy()
train_data['label'] = train_data['sentiment'].map(label_mapping)

# Do the same for the test data
test_data = test_df[['text', 'sentiment']].copy()
test_data['label'] = test_data['sentiment'].map(label_mapping)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the training data into training and validation sets while preserving label distribution
train_split, val_split = train_test_split(
    train_data, 
    test_size=0.2, 
    stratify=train_data['label'], 
    random_state=42
)


In [ ]:
from torch.utils.data import Dataset
import torch

class TextClassificationDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, max_len=128):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        sample_text = str(self.inputs[index])
        sample_label = int(self.targets[index])

        tokenized = self.tokenizer(
            sample_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'label': torch.tensor(sample_label, dtype=torch.long)
        }


In [ ]:
# Set computation device to GPU if available, otherwise fallback to CPU
compute_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
compute_device


device(type='cuda')

In [ ]:
from transformers import AutoTokenizer

# Define the pretrained model name and load its tokenizer
pretrained_model = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

# Initialize datasets using the custom dataset class
train_set = TextClassificationDataset(
    inputs=train_split["text"].tolist(),
    targets=train_split["label"].tolist(),
    tokenizer=tokenizer
)

val_set = TextClassificationDataset(
    inputs=val_split["text"].tolist(),
    targets=val_split["label"].tolist(),
    tokenizer=tokenizer
)

test_set = TextClassificationDataset(
    inputs=test_data["text"].tolist(),
    targets=test_data["label"].tolist(),
    tokenizer=tokenizer
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

# Specify the name of the pretrained BERT model and load its tokenizer
model_identifier = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_identifier)

# Construct dataset objects for training, validation, and testing
training_data = TextClassificationDataset(
    inputs=train_split["text"].tolist(),
    targets=train_split["label"].tolist(),
    tokenizer=tokenizer
)

validation_data = TextClassificationDataset(
    inputs=val_split["text"].tolist(),
    targets=val_split["label"].tolist(),
    tokenizer=tokenizer
)

testing_data = TextClassificationDataset(
    inputs=test_data["text"].tolist(),
    targets=test_data["label"].tolist(),
    tokenizer=tokenizer
)


In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# Load the pretrained model for sequence classification with 3 labels
classification_model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model,
    num_labels=3,
    problem_type="single_label_classification"
)
classification_model.to(compute_device)

# Set training hyperparameters
num_epochs = 5
lr_rate = 3e-5
l2_reg = 0.01

# Initialize the AdamW optimizer with weight decay
optimizer = AdamW(classification_model.parameters(), lr=lr_rate, weight_decay=l2_reg)

# Calculate total steps and warmup steps
training_steps = len(train_loader) * num_epochs
warmup_steps = len(train_loader) // 10  # 10% of steps for warmup phase

# Setup learning rate scheduler with warmup
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=training_steps
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def train_one_epoch(model, train_loader, optimizer, scheduler, device):
    """Perform training for a single epoch"""
    model.train()
    total_loss = 0.0
    progress_freq = max(1, len(train_loader) // 5)  # Update progress every 20% of batches

    for step, batch in enumerate(train_loader):
        # Move batch data to the specified device (GPU/CPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['label'].to(device)

        # Reset gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=targets
        )

        loss = outputs.loss
        total_loss += loss.item()

        # Perform backward pass
        loss.backward()

        # Apply gradient clipping to avoid gradient explosion
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update model parameters
        optimizer.step()
        scheduler.step()

        # Display progress
        if (step + 1) % progress_freq == 0:
            print(f"Step {step+1}/{len(train_loader)} | Loss: {loss.item():.4f}")

    # Return the average loss for the epoch
    avg_epoch_loss = total_loss / len(train_loader)
    return avg_epoch_loss

def evaluate_performance(model, val_loader, device):
    """Evaluate model performance on the validation set"""
    model.eval()
    total_val_loss = 0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for batch in val_loader:
            # Move batch data to the specified device (GPU/CPU)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Perform forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            total_val_loss += loss.item()

            # Extract model predictions
            logits = outputs.logits
            _, predicted = torch.max(logits, dim=1)

            # Store the true and predicted labels
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    # Calculate the average validation loss
    avg_val_loss = total_val_loss / len(val_loader)

    # Calculate accuracy
    accuracy = np.mean(np.array(predicted_labels) == np.array(true_labels))

    # Calculate precision, recall, and F1-score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, predicted_labels, average=None, zero_division=0
    )

    return avg_val_loss, accuracy, precision, recall, f1, true_labels, predicted_labels


In [ ]:
# Define sentiment labels and initialize history tracking dictionary
sentiment_labels = ['negative', 'neutral', 'positive']
training_history = {
    'train_loss': [],
    'val_loss': [],
    'test_loss': [],
    'val_accuracy': [],
    'test_accuracy': [],
    'val_precision': {label: [] for label in sentiment_labels},
    'val_recall': {label: [] for label in sentiment_labels},
    'val_f1': {label: [] for label in sentiment_labels}
}

# Monitor GPU memory usage if CUDA is available
if torch.cuda.is_available():
    training_history['gpu_memory_usage'] = []


In [ ]:
# Training loop over multiple epochs
for current_epoch in range(num_epochs):
    print(f"\nEpoch {current_epoch+1}/{num_epochs}")
    print("-" * 50)

    # Train the model for one epoch
    train_epoch_loss = train_one_epoch(model, train_loader, optimizer, scheduler, compute_device)
    training_history['train_loss'].append(train_epoch_loss)

    # Monitor GPU memory usage if CUDA is available
    if torch.cuda.is_available():
        allocated_memory = torch.cuda.memory_allocated(0) / 1e9  # Convert to GB
        training_history['gpu_memory_usage'].append(allocated_memory)
        print(f"GPU Memory Usage: {allocated_memory:.2f} GB")

    # Evaluate the model on the validation set
    val_epoch_loss, val_acc, val_precision, val_recall, val_f1, _, _ = evaluate_performance(model, val_loader, compute_device)
    training_history['val_loss'].append(val_epoch_loss)
    training_history['val_accuracy'].append(val_acc)

    # Evaluate the model on the test set
    test_epoch_loss, test_acc, _, _, _, _, _ = evaluate_performance(model, test_loader, compute_device)
    training_history['test_loss'].append(test_epoch_loss)
    training_history['test_accuracy'].append(test_acc)

    # Store per-class precision, recall, and F1-score
    for idx, label in enumerate(sentiment_labels):
        if idx < len(val_precision):
            training_history['val_precision'][label].append(val_precision[idx])
            training_history['val_recall'][label].append(val_recall[idx])
            training_history['val_f1'][label].append(val_f1[idx])

    # Print summary for the current epoch
    print(f"Training Loss: {train_epoch_loss:.4f}")
    print(f"Validation Loss: {val_epoch_loss:.4f} | Validation Accuracy: {val_acc:.4f}")
    print(f"Test Loss: {test_epoch_loss:.4f} | Test Accuracy: {test_acc:.4f}")

    # Print detailed metrics for each class
    print("\nPer-Class Validation Metrics:")
    print("-" * 50)
    print(f"{'Class':<15} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}")
    print("-" * 50)
    for idx, label in enumerate(sentiment_labels):
        if idx < len(val_precision):
            print(f"{label:<15} {val_precision[idx]:.4f}{' ':6} {val_recall[idx]:.4f}{' ':6} {val_f1[idx]:.4f}")
    print("-" * 50)



Epoch 1/5
--------------------------------------------------
Batch 110/550 | Loss: 0.7213
Batch 220/550 | Loss: 0.4257
Batch 330/550 | Loss: 0.6635
Batch 440/550 | Loss: 0.3573
Batch 550/550 | Loss: 0.5038
GPU Memory Usage: 1.78 GB
Training Loss: 0.5653
Validation Loss: 0.5335 | Validation Accuracy: 0.7833
Test Loss: 0.5276 | Test Accuracy: 0.7838

Per-class Validation Metrics:
--------------------------------------------------
Class           Precision  Recall     F1-Score  
--------------------------------------------------
negative        0.7284       0.8740       0.7946
neutral         0.8025       0.6707       0.7307
positive        0.8210       0.8468       0.8337
--------------------------------------------------

Epoch 2/5
--------------------------------------------------
Batch 110/550 | Loss: 0.2363
Batch 220/550 | Loss: 0.2025
Batch 330/550 | Loss: 0.3218
Batch 440/550 | Loss: 0.4083
Batch 550/550 | Loss: 0.7546
GPU Memory Usage: 1.78 GB
Training Loss: 0.3975
Validation Los

In [ ]:
# Set the model to evaluation mode
model.eval()

# Evaluate the model on the test set
_, _, test_precision, test_recall, test_f1, true_labels, predicted_labels = evaluate_performance(model, test_loader, compute_device)

# Print the final classification report for the test set
print("Final Classification Report on Test Set:")
print(classification_report(true_labels, predicted_labels, target_names=sentiment_labels))


Final Classification Report on Test Set:
              precision    recall  f1-score   support

    negative       0.78      0.80      0.79      1001
     neutral       0.75      0.75      0.75      1430
    positive       0.84      0.82      0.83      1103

    accuracy                           0.78      3534
   macro avg       0.79      0.79      0.79      3534
weighted avg       0.78      0.78      0.78      3534



In [ ]:
# Sample texts for prediction
sample_texts = [
    "Its absolutely amazing!",
    "Nothing much to say. Okay okayish feel.",
    "I regret about my decision."
]

# Display predictions for each text
print(f"{'Text':<50} {'Predicted Sentiment'}")
print("-" * 75)

for input_text in sample_texts:
    model.eval()

    # Tokenize the input text
    encoded_input = tokenizer(
        input_text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )

    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Perform inference without updating gradients
    with torch.no_grad():
        model_outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.nn.functional.softmax(model_outputs.logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1)

    # Map prediction to sentiment label
    sentiment_mapping = {0: "negative", 1: "neutral", 2: "positive"}
    predicted_sentiment = sentiment_mapping[predicted_class.item()]
    
    # Print the result
    print(f"{input_text:<70} {predicted_sentiment}")


Text                                               Prediction
---------------------------------------------------------------------------
Its absolutely amazing!                                                positive
Nothing much to say. Okay okayish feel.                                neutral
I regret about my decision.                                            negative
